This project is aimed at better understanding the types of customers who purchase chips and their purchasing behaviour within the region and provide recommendations for commercial application

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
trans = pd.read_excel('QVI_transaction_data.xlsx')
purchase = pd.read_csv('QVI_purchase_behaviour.csv')

In [ ]:
trans.head()

In [ ]:
trans.columns = map(str.lower, trans.columns)
trans.columns

In [ ]:
trans['prod_size "g"'] = trans['prod_name'].str.strip(r'\d+')
trans['prod_size "g"'].replace(r'[A-Za-z\D+]', '', regex=True, inplace=True)
trans['prod_name'].replace(r'\d+\w', '', regex=True, inplace=True)
trans['prod_name'] = trans['prod_name'].str.strip()
trans.head()